In [4]:
import pandas as pd
import pandas_datareader as pdr
import yfinance as yf
import numpy as np
from datetime import datetime
from datetime import timedelta
import backtrader as bt
import pyfolio as pf
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

In [ ]:
def load_data(ticker_ls, start_date, end_date):
    df = pd.DataFrame(columns = ticker_ls)
    
    for ticker in ticker_ls:
        df[ticker] = yf.download(ticker, start = start_date, end = end_date)["Adj Close"]
        
    return df

In [ ]:
# 13612W filter: average annual returns over the past 1, 3, 6, 12 months.
def cal_momentum(df, ticker):
    p0 = df[ticker]
    
    # working days
    try:
        p1 = df[ticker].shift(20)
        p3 = df[ticker].shift(60)
        p6 = df[ticker].shift(120)
        p12 = df[ticker].shift(240)
        momentum = 12*(p0/p1-1) + 4*(p0/p3-1) + 2*(p0/p6-1) + 1*(p0/p12-1)
    
    except:
        pass
    
    return momentum

In [ ]:
def select_asset(x):
    selected_df = pd.Series([0, 0], index=['selected', 'price'])
    
    # 
    if x['SPY_mom'] > 0 and x['EFA_mom'] > 0 and x['EEM_mom'] > 0 and x['AGG_mom'] > 0:
        selected_mom = max(x['SPY_mom'], x['EFA_mom'], x['EEM_mom'], x['AGG_mom'])
        
    else:
        selected_mom = max(x['SHY_mom'], x['IEF_mom'], x['LQD_mom'])
    
    # remove '_mom' from the seleceted asset name
    selected_df['selected'] = x[x==selected_mom].index[0][:3]
    selected_df['price'] = x[selected_df['selected']]
    
    return selected_df